<a href="https://colab.research.google.com/github/thedatasense/llm-healthcare/blob/main/MIMIC_GPT_Evaluation_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q  sqlalchemy cockroachdb pandas psycopg2-binary matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.8 MB/s eta 0:00:00


In [ ]:
!curl --create-dirs -o $HOME/.postgresql/root.crt 'https://cockroachlabs.cloud/clusters/5bbbe91d-b65e-410e-a783-597c93f501f6/cert'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2728    0  2728    0     0  13442      0 --:--:-- --:--:-- --:--:-- 13438


In [ ]:
import pandas as pd
from IPython.display import clear_output
from sqlalchemy.engine import create_engine
from google.colab import userdata
engine = create_engine(userdata.get('DB_URL'))
#from datasets import load_dataset
from openai import OpenAI
import io
import base64
import random
import requests
import torch
from PIL import Image
#from transformers import AutoProcessor,Qwen2_5_VLForConditionalGeneration
#from qwen_vl_utils import process_vision_info
import os
import pandas as pd
from sqlalchemy.engine import create_engine
from transformers import AutoProcessor, BitsAndBytesConfig
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/thedatasense/llm-healthcare.git

Cloning into 'llm-healthcare'...
remote: Enumerating objects: 1339, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 1339 (delta 111), reused 106 (delta 53), pack-reused 1168 (from 2)
Receiving objects: 100% (1339/1339), 34.88 MiB | 21.31 MiB/s, done.
Resolving deltas: 100% (275/275), done.
Updating files: 100% (1385/1385), done.


In [ ]:
def insert_model_response(engine, uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link):
    from sqlalchemy import text
    with engine.connect() as conn:
        trans = conn.begin()
        try:
            conn.execute(text("""
                INSERT INTO model_responses_r2
                (uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link)
                VALUES (:uid,:question_id,:question, :question_category, :actual_answer, :model_name, :model_answer, :image_link)
            """), {
                "uid": uid,
                "question_id": question_id,
                "question": question,
                "question_category": question_category,
                "actual_answer": actual_answer,
                "model_name": model_name,
                "model_answer": model_answer,
                "image_link": image_link
            })
            trans.commit()  # Commit the transaction
        except Exception as e:
            trans.rollback()
            raise e



In [ ]:
def check_duplicate(engine,uid,question_id,question, question_category, model_name,image_link):
    query = text("""
        SELECT 1 FROM model_responses_r2
        WHERE
        uid = :uid
        AND question_id = :question_id and
        question = :question
          AND question_category = :question_category
          AND model_name = :model_name
          AND image_link = :image_link
        LIMIT 1
    """)
    with engine.connect() as conn:
        result = conn.execute(query, {
            "uid": uid,
            "question_id": question_id,
            "question": question,
            "question_category": question_category,
            "model_name": model_name,
            "image_link": image_link
        }).fetchone()
    return result is not None

In [ ]:
def fetch_generation_data(engine):
    import pandas as pd
    import re
    from sqlalchemy import text
    from sqlalchemy.dialects.postgresql.base import PGDialect
    def fake_get_server_version_info(self, connection):
        version_str = connection.execute(text("SELECT version()")).scalar()
        match = re.search(r'v(\d+)\.(\d+)\.(\d+)', version_str)
        if match:
            return tuple(map(int, match.groups()))
        return (13, 0, 0)
    PGDialect._get_server_version_info = fake_get_server_version_info
    query = f"SELECT id,question_id,condition as question_type, text as question,answer as ground_truth,image from mimic_all_qns; "
    return pd.read_sql(query, con=engine)



In [ ]:
oai_key=userdata.get('OPENAI_SAIL')

In [ ]:
def encode_image_stream(image_path):
    try:
        if not os.path.exists(image_path):
            error_file_list.add(image_path)
            return None

        with open(image_path, "rb") as img_file:
            return base64.b64encode(img_file.read()).decode("utf-8")
    except Exception as e:
        error_file_list.add(image_path)
        return None

def generate_gpt_response(prompt_text, image_link):
    client = OpenAI(api_key=oai_key)
    base64_image = encode_image_stream(image_link)

    if base64_image is None:
        return f"Unable to process image: {image_link}"

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an expert medical professional. "
                    "When responding, provide a concise explanation of the image findings. "
                    "For example, if asked about abnormalities, answer briefly with terms like 'atelectasis, lung opacity'."
                )
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt_text,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                    },
                ],
            },
        ],
    )
    return response.choices[0].message.content

In [ ]:
source_folder='/content/drive/MyDrive/Health_Data/MIMIC_JPG_AVL/mimic-cxr-jpg/2.1.0/files/'

In [ ]:
import random
from IPython.display import Image, display

def print_a_sample(df,prompt_prefix,source_folder=source_folder,width=250):
    # Get a random row as a Series
    row = df.sample(n=1).iloc[0]  # Add .iloc[0] to get the row as a Series

    # Now these will return scalar values
    question = row["question"]
    actual_answer = row["ground_truth"]
    image_link = source_folder + row["image"]

    print(f"{question}")
    print(f"GT: {actual_answer}")
    print(f"Image: {image_link}")


    # Uncommented to display the image
    display(Image(filename=image_link, width=width))
    generated_answer = generate_gpt_response(prompt_prefix + " " + row["question_type"] + ":" + row["question"] + prompt_prefix, image_link)
    print(f"gpt-4o : {generated_answer}")
    print('--------------------------------')


In [ ]:
from sqlalchemy import text
import time
error_file_list = set()
model_id = "gpt-4o"

for index, row in fetch_generation_data(engine).iterrows():
    uid = row["id"]
    question_id = row["question_id"]
    question_category = row["question_type"]
    question = row["question"]
    actual_answer = row["ground_truth"]
    image_link = source_folder + row["image"]

    if check_duplicate(engine, uid, str(question_id), question, question_category, model_id, image_link):
        clear_output(wait=True)
        continue

    try:
        generated_answer = generate_gpt_response(question, image_link)

        if not generated_answer.startswith("Unable to process image"):
            insert_model_response(
                engine, uid, question_id, question, question_category,
                actual_answer, model_id, generated_answer, image_link
            )
    except Exception as e:
        error_file_list.add(image_link)
        continue

    time.sleep(5)
